In [24]:
import os
import pandas as pd
from tqdm import tqdm
import datetime
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.tokenize import sent_tokenize, RegexpTokenizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import string

import plotly.express as px
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/drknt_rises/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/drknt_rises/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/drknt_rises/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [25]:
current_path = os.getcwd()
current_path = current_path.split('movie-pre-buzz-to-box-office')
path = current_path[0] + 'movie-pre-buzz-to-box-office'
path

'/Users/drknt_rises/Desktop/Personal/Git Repos/movie-pre-buzz-to-box-office'

In [26]:
video_description = pd.read_csv(path+'/Data/Video Description/videoDescription.csv').drop_duplicates()
video_description['publishedAt'] = pd.to_datetime(video_description['publishedAt'])

iden_tpoics = pd.read_csv(path+'/Data/Topic Modelling/iden_topics.csv')
topic_classification = pd.read_csv(path+'/Data/Topic Modelling/topic_classification.csv')

In [27]:
comments = pd.read_csv(path+'/Data/Comments/Video Comments.csv', sep='|', lineterminator='\n').drop_duplicates()
comments['publishedAt'] = pd.to_datetime(comments['publishedAt'])
comments = comments.loc[comments['code'] == 200]
comments = comments.rename(columns={'publishedAt': 'postedAt'})
comments

,videoId,code,error_reason,id,replies,comment,postedAt
1,j5BnpaiMvO4,200,NaN,UgxxDh7BZywVmYQ-cKl4AaABAg,0.0,One of the best movies I’ve seen everr!!! Cami...,2022-07-03 08:53:31+00:00
2,j5BnpaiMvO4,200,NaN,UgzGKGwt-KVW_IRQcix4AaABAg,0.0,Looks intense. Thanks for all the comments… gu...,2022-06-20 22:57:22+00:00
3,j5BnpaiMvO4,200,NaN,Ugzdy66Y0udge3yasId4AaABAg,0.0,Song? Plssssss,2020-08-29 23:37:17+00:00
4,j5BnpaiMvO4,200,NaN,Ugwm3Mlcm5H2lesS0fl4AaABAg,0.0,You'll just watch a father being a burden to h...,2020-07-14 04:39:10+00:00
5,j5BnpaiMvO4,200,NaN,UgwZpjmZjZXpQ6Blf494AaABAg,2.0,Wheres the bear? Why is he a bear 🐻 🐻‍❄? He is...,2020-07-13 14:41:10+00:00
...,...,...,...,...,...,...,...
369182,fm_zzAMuvyQ,200,NaN,UgyvGQpXK1oRKW0BvZZ4AaABAg,3.0,where to watch this movie...please tell me Buy...,2019-11-13 20:25:03+00:00
369183,fm_zzAMuvyQ,200,NaN,UgyMoXQjz4fYRzbe31x4AaABAg,1.0,gaaaaaaaaaaaaaay! wefa3rf wa,2018-05-07 03:40:04+00:00
369184,fm_zzAMuvyQ,200,NaN,Ugzsz2CNGeKIO1T3OCF4AaABAg,3.0,Took my husband to this movie. Great faith and...,2018-03-29 19:57:52+00:00
369185,fm_zzAMuvyQ,200,NaN,Ugw1RVGlpw2O_wM1Zo14AaABAg,1.0,Must see...Can't miss...I can only imagine! Da...,2018-03-16 02:23:20+00:00


In [28]:
video_description = video_description[['videoId', 'publishedAt']]
comments = comments.merge(video_description, left_on='videoId', right_on='videoId', how='left')
comments['cutoffdate'] = comments['publishedAt'] + datetime.timedelta(days=21)
comments = comments.loc[comments['postedAt'] < comments['cutoffdate']]
comments

,videoId,code,error_reason,id,replies,comment,postedAt,publishedAt,cutoffdate
17,j5BnpaiMvO4,200,NaN,UgyC1N24gUVO95q8SW54AaABAg,3.0,Looks boring :( and even the lead actress look...,2019-10-19 18:14:05+00:00,2019-10-09 09:22:08+00:00,2019-10-30 09:22:08+00:00
18,j5BnpaiMvO4,200,NaN,UgyWcqppkxY1mLGyqNZ4AaABAg,0.0,Things can go so bad🙈,2019-10-10 10:56:20+00:00,2019-10-09 09:22:08+00:00,2019-10-30 09:22:08+00:00
19,j5BnpaiMvO4,200,NaN,UgzXS0zcfdGWiYBbulx4AaABAg,0.0,Masha and the bear spin off???,2019-10-09 14:50:02+00:00,2019-10-09 09:22:08+00:00,2019-10-30 09:22:08+00:00
20,j5BnpaiMvO4,200,NaN,UgwTZh0s-v6ta3TbAfx4AaABAg,10.0,Sometime I think what kind of people watch the...,2019-10-09 09:56:27+00:00,2019-10-09 09:22:08+00:00,2019-10-30 09:22:08+00:00
21,j5BnpaiMvO4,200,NaN,Ugw0TALvQXFY_iHYFjJ4AaABAg,2.0,boring another melodrama nonsense and worst of...,2019-10-09 09:54:28+00:00,2019-10-09 09:22:08+00:00,2019-10-30 09:22:08+00:00
...,...,...,...,...,...,...,...,...,...
369118,nrXlY6gzTTM,200,NaN,UgxEeisvPIaQ0bJuYnx4AaABAg,0.0,Divergent music,2017-11-08 05:36:54+00:00,2017-11-08 05:31:00+00:00,2017-11-29 05:31:00+00:00
369119,nrXlY6gzTTM,200,NaN,Ugyu9iaLbJv51WaQWvt4AaABAg,1.0,Xmen and Fantastic Four IN MCU PLEASE FOX Get ...,2017-11-08 05:35:48+00:00,2017-11-08 05:31:00+00:00,2017-11-29 05:31:00+00:00
369120,nrXlY6gzTTM,200,NaN,UgzOoFffvhW2xSdfHJZ4AaABAg,1.0,TOM AND MERYL Facu Auci AND STEVEN,2017-11-08 05:31:47+00:00,2017-11-08 05:31:00+00:00,2017-11-29 05:31:00+00:00
369121,nrXlY6gzTTM,200,NaN,Ugw63wCEwfFTPiu8Pwp4AaABAg,0.0,Spielberg FTW!!!,2017-11-08 05:31:42+00:00,2017-11-08 05:31:00+00:00,2017-11-29 05:31:00+00:00


In [29]:
def replace_punct(text):
    text = str(text)
    for i in list(string.punctuation):
        text = text.replace(i, ' ')
    text = text.lower()
    return text

comments['comment'] = comments['comment'].apply(lambda row: str(row).lower())
stop_words = stopwords.words('english')
comments['comment'] = comments['comment'].apply(lambda row: ' '.join([word for word in row.split() if word not in stop_words]))
comments['comment'] = comments['comment'].apply(lambda row: replace_punct(row))
comments

,videoId,code,error_reason,id,replies,comment,postedAt,publishedAt,cutoffdate
17,j5BnpaiMvO4,200,NaN,UgyC1N24gUVO95q8SW54AaABAg,3.0,looks boring even lead actress looks bored ...,2019-10-19 18:14:05+00:00,2019-10-09 09:22:08+00:00,2019-10-30 09:22:08+00:00
18,j5BnpaiMvO4,200,NaN,UgyWcqppkxY1mLGyqNZ4AaABAg,0.0,things go bad🙈,2019-10-10 10:56:20+00:00,2019-10-09 09:22:08+00:00,2019-10-30 09:22:08+00:00
19,j5BnpaiMvO4,200,NaN,UgzXS0zcfdGWiYBbulx4AaABAg,0.0,masha bear spin off,2019-10-09 14:50:02+00:00,2019-10-09 09:22:08+00:00,2019-10-30 09:22:08+00:00
20,j5BnpaiMvO4,200,NaN,UgwTZh0s-v6ta3TbAfx4AaABAg,10.0,sometime think kind people watch type movie b...,2019-10-09 09:56:27+00:00,2019-10-09 09:22:08+00:00,2019-10-30 09:22:08+00:00
21,j5BnpaiMvO4,200,NaN,Ugw0TALvQXFY_iHYFjJ4AaABAg,2.0,boring another melodrama nonsense worst bear l...,2019-10-09 09:54:28+00:00,2019-10-09 09:22:08+00:00,2019-10-30 09:22:08+00:00
...,...,...,...,...,...,...,...,...,...
369118,nrXlY6gzTTM,200,NaN,UgxEeisvPIaQ0bJuYnx4AaABAg,0.0,divergent music,2017-11-08 05:36:54+00:00,2017-11-08 05:31:00+00:00,2017-11-29 05:31:00+00:00
369119,nrXlY6gzTTM,200,NaN,Ugyu9iaLbJv51WaQWvt4AaABAg,1.0,xmen fantastic four mcu please fox get life lo...,2017-11-08 05:35:48+00:00,2017-11-08 05:31:00+00:00,2017-11-29 05:31:00+00:00
369120,nrXlY6gzTTM,200,NaN,UgzOoFffvhW2xSdfHJZ4AaABAg,1.0,tom meryl facu auci steven,2017-11-08 05:31:47+00:00,2017-11-08 05:31:00+00:00,2017-11-29 05:31:00+00:00
369121,nrXlY6gzTTM,200,NaN,Ugw63wCEwfFTPiu8Pwp4AaABAg,0.0,spielberg ftw,2017-11-08 05:31:42+00:00,2017-11-08 05:31:00+00:00,2017-11-29 05:31:00+00:00


In [30]:
comments['comment'] = comments['comment'].apply(lambda row: sent_tokenize(row))
comments['comment'] = comments['comment'].apply(lambda row: ' '.join(row))
comments

,videoId,code,error_reason,id,replies,comment,postedAt,publishedAt,cutoffdate
17,j5BnpaiMvO4,200,NaN,UgyC1N24gUVO95q8SW54AaABAg,3.0,looks boring even lead actress looks bored ...,2019-10-19 18:14:05+00:00,2019-10-09 09:22:08+00:00,2019-10-30 09:22:08+00:00
18,j5BnpaiMvO4,200,NaN,UgyWcqppkxY1mLGyqNZ4AaABAg,0.0,things go bad🙈,2019-10-10 10:56:20+00:00,2019-10-09 09:22:08+00:00,2019-10-30 09:22:08+00:00
19,j5BnpaiMvO4,200,NaN,UgzXS0zcfdGWiYBbulx4AaABAg,0.0,masha bear spin off,2019-10-09 14:50:02+00:00,2019-10-09 09:22:08+00:00,2019-10-30 09:22:08+00:00
20,j5BnpaiMvO4,200,NaN,UgwTZh0s-v6ta3TbAfx4AaABAg,10.0,sometime think kind people watch type movie b...,2019-10-09 09:56:27+00:00,2019-10-09 09:22:08+00:00,2019-10-30 09:22:08+00:00
21,j5BnpaiMvO4,200,NaN,Ugw0TALvQXFY_iHYFjJ4AaABAg,2.0,boring another melodrama nonsense worst bear l...,2019-10-09 09:54:28+00:00,2019-10-09 09:22:08+00:00,2019-10-30 09:22:08+00:00
...,...,...,...,...,...,...,...,...,...
369118,nrXlY6gzTTM,200,NaN,UgxEeisvPIaQ0bJuYnx4AaABAg,0.0,divergent music,2017-11-08 05:36:54+00:00,2017-11-08 05:31:00+00:00,2017-11-29 05:31:00+00:00
369119,nrXlY6gzTTM,200,NaN,Ugyu9iaLbJv51WaQWvt4AaABAg,1.0,xmen fantastic four mcu please fox get life loser,2017-11-08 05:35:48+00:00,2017-11-08 05:31:00+00:00,2017-11-29 05:31:00+00:00
369120,nrXlY6gzTTM,200,NaN,UgzOoFffvhW2xSdfHJZ4AaABAg,1.0,tom meryl facu auci steven,2017-11-08 05:31:47+00:00,2017-11-08 05:31:00+00:00,2017-11-29 05:31:00+00:00
369121,nrXlY6gzTTM,200,NaN,Ugw63wCEwfFTPiu8Pwp4AaABAg,0.0,spielberg ftw,2017-11-08 05:31:42+00:00,2017-11-08 05:31:00+00:00,2017-11-29 05:31:00+00:00


In [31]:
sent = SentimentIntensityAnalyzer()
comments['score'] = comments['comment'].apply(lambda row: sent.polarity_scores(row))

comments['negative'] = comments['score'].apply(lambda row: row['neg'])
comments['neutral'] = comments['score'].apply(lambda row: row['neu'])
comments['positive'] = comments['score'].apply(lambda row: row['pos'])
comments['compound'] = comments['score'].apply(lambda row: row['compound'])

comments

,videoId,code,error_reason,id,replies,comment,postedAt,publishedAt,cutoffdate,score,negative,neutral,positive,compound
17,j5BnpaiMvO4,200,NaN,UgyC1N24gUVO95q8SW54AaABAg,3.0,looks boring even lead actress looks bored ...,2019-10-19 18:14:05+00:00,2019-10-09 09:22:08+00:00,2019-10-30 09:22:08+00:00,"{'neg': 0.228, 'neu': 0.379, 'pos': 0.393, 'co...",0.228,0.379,0.393,0.8208
18,j5BnpaiMvO4,200,NaN,UgyWcqppkxY1mLGyqNZ4AaABAg,0.0,things go bad🙈,2019-10-10 10:56:20+00:00,2019-10-09 09:22:08+00:00,2019-10-30 09:22:08+00:00,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000
19,j5BnpaiMvO4,200,NaN,UgzXS0zcfdGWiYBbulx4AaABAg,0.0,masha bear spin off,2019-10-09 14:50:02+00:00,2019-10-09 09:22:08+00:00,2019-10-30 09:22:08+00:00,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000
20,j5BnpaiMvO4,200,NaN,UgwTZh0s-v6ta3TbAfx4AaABAg,10.0,sometime think kind people watch type movie b...,2019-10-09 09:56:27+00:00,2019-10-09 09:22:08+00:00,2019-10-30 09:22:08+00:00,"{'neg': 0.12, 'neu': 0.537, 'pos': 0.343, 'com...",0.120,0.537,0.343,0.8957
21,j5BnpaiMvO4,200,NaN,Ugw0TALvQXFY_iHYFjJ4AaABAg,2.0,boring another melodrama nonsense worst bear l...,2019-10-09 09:54:28+00:00,2019-10-09 09:22:08+00:00,2019-10-30 09:22:08+00:00,"{'neg': 0.397, 'neu': 0.513, 'pos': 0.09, 'com...",0.397,0.513,0.090,-0.8625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369118,nrXlY6gzTTM,200,NaN,UgxEeisvPIaQ0bJuYnx4AaABAg,0.0,divergent music,2017-11-08 05:36:54+00:00,2017-11-08 05:31:00+00:00,2017-11-29 05:31:00+00:00,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000
369119,nrXlY6gzTTM,200,NaN,Ugyu9iaLbJv51WaQWvt4AaABAg,1.0,xmen fantastic four mcu please fox get life loser,2017-11-08 05:35:48+00:00,2017-11-08 05:31:00+00:00,2017-11-29 05:31:00+00:00,"{'neg': 0.222, 'neu': 0.392, 'pos': 0.386, 'co...",0.222,0.392,0.386,0.3612
369120,nrXlY6gzTTM,200,NaN,UgzOoFffvhW2xSdfHJZ4AaABAg,1.0,tom meryl facu auci steven,2017-11-08 05:31:47+00:00,2017-11-08 05:31:00+00:00,2017-11-29 05:31:00+00:00,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000
369121,nrXlY6gzTTM,200,NaN,Ugw63wCEwfFTPiu8Pwp4AaABAg,0.0,spielberg ftw,2017-11-08 05:31:42+00:00,2017-11-08 05:31:00+00:00,2017-11-29 05:31:00+00:00,"{'neg': 0.0, 'neu': 0.25, 'pos': 0.75, 'compou...",0.000,0.250,0.750,0.4588


In [32]:
def polarity_score(compound):
    if compound > 0.05:
        return "positive"
    elif compound < -0.5:
        return "negative"
    elif compound >= -0.05 and compound < 0.05:
        return "neutral"

comments['sentiment'] = comments['compound'].apply(lambda val: polarity_score(val))
comments

,videoId,code,error_reason,id,replies,comment,postedAt,publishedAt,cutoffdate,score,negative,neutral,positive,compound,sentiment
17,j5BnpaiMvO4,200,NaN,UgyC1N24gUVO95q8SW54AaABAg,3.0,looks boring even lead actress looks bored ...,2019-10-19 18:14:05+00:00,2019-10-09 09:22:08+00:00,2019-10-30 09:22:08+00:00,"{'neg': 0.228, 'neu': 0.379, 'pos': 0.393, 'co...",0.228,0.379,0.393,0.8208,positive
18,j5BnpaiMvO4,200,NaN,UgyWcqppkxY1mLGyqNZ4AaABAg,0.0,things go bad🙈,2019-10-10 10:56:20+00:00,2019-10-09 09:22:08+00:00,2019-10-30 09:22:08+00:00,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,neutral
19,j5BnpaiMvO4,200,NaN,UgzXS0zcfdGWiYBbulx4AaABAg,0.0,masha bear spin off,2019-10-09 14:50:02+00:00,2019-10-09 09:22:08+00:00,2019-10-30 09:22:08+00:00,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,neutral
20,j5BnpaiMvO4,200,NaN,UgwTZh0s-v6ta3TbAfx4AaABAg,10.0,sometime think kind people watch type movie b...,2019-10-09 09:56:27+00:00,2019-10-09 09:22:08+00:00,2019-10-30 09:22:08+00:00,"{'neg': 0.12, 'neu': 0.537, 'pos': 0.343, 'com...",0.120,0.537,0.343,0.8957,positive
21,j5BnpaiMvO4,200,NaN,Ugw0TALvQXFY_iHYFjJ4AaABAg,2.0,boring another melodrama nonsense worst bear l...,2019-10-09 09:54:28+00:00,2019-10-09 09:22:08+00:00,2019-10-30 09:22:08+00:00,"{'neg': 0.397, 'neu': 0.513, 'pos': 0.09, 'com...",0.397,0.513,0.090,-0.8625,negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369118,nrXlY6gzTTM,200,NaN,UgxEeisvPIaQ0bJuYnx4AaABAg,0.0,divergent music,2017-11-08 05:36:54+00:00,2017-11-08 05:31:00+00:00,2017-11-29 05:31:00+00:00,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,neutral
369119,nrXlY6gzTTM,200,NaN,Ugyu9iaLbJv51WaQWvt4AaABAg,1.0,xmen fantastic four mcu please fox get life loser,2017-11-08 05:35:48+00:00,2017-11-08 05:31:00+00:00,2017-11-29 05:31:00+00:00,"{'neg': 0.222, 'neu': 0.392, 'pos': 0.386, 'co...",0.222,0.392,0.386,0.3612,positive
369120,nrXlY6gzTTM,200,NaN,UgzOoFffvhW2xSdfHJZ4AaABAg,1.0,tom meryl facu auci steven,2017-11-08 05:31:47+00:00,2017-11-08 05:31:00+00:00,2017-11-29 05:31:00+00:00,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,neutral
369121,nrXlY6gzTTM,200,NaN,Ugw63wCEwfFTPiu8Pwp4AaABAg,0.0,spielberg ftw,2017-11-08 05:31:42+00:00,2017-11-08 05:31:00+00:00,2017-11-29 05:31:00+00:00,"{'neg': 0.0, 'neu': 0.25, 'pos': 0.75, 'compou...",0.000,0.250,0.750,0.4588,positive


In [33]:
comments_ = comments.groupby('videoId').agg(
    positive_sent = pd.NamedAgg('sentiment', aggfunc=lambda x: (x == 'positive').sum()),
    negative_sent = pd.NamedAgg('sentiment', aggfunc=lambda x: (x == 'negative').sum()),
    neutral_sent = pd.NamedAgg('sentiment', aggfunc=lambda x: (x == 'neutral').sum()),
    total_comments = pd.NamedAgg('videoId', aggfunc='count')
).reset_index(drop=False)

comments_['pos-neg'] = np.log(comments_['positive_sent']) / np.log(comments_['negative_sent'])
comments_

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning:

divide by zero encountered in log



,videoId,positive_sent,negative_sent,neutral_sent,total_comments,pos-neg
0,-0FU8z_1Arc,25,3,17,48,2.929947
1,-3T78VfGyes,1,0,0,1,-0.000000
2,-9-HBqVbtTo,51,7,24,92,2.020559
3,-Denciie5oA,430,256,441,1291,1.093524
4,-LtCIImfSCk,244,42,145,476,1.470747
...,...,...,...,...,...,...
436,zRaUIPMq-u0,7,2,4,16,2.807355
437,zbwL8YMdQUE,5,0,2,7,-0.000000
438,zkHP4RXHzBg,9,1,3,14,inf
439,zo2V5CQMuJw,196,22,97,328,1.707552


In [34]:
video_description = pd.read_csv(path+'/Data/Video Description/videoDescription.csv').drop_duplicates()
video_description['publishedAt'] = pd.to_datetime(video_description['publishedAt'])

topic_classification = topic_classification.merge(iden_tpoics[['Topic', 'CustomName']], left_on='Topic', right_on='Topic', how='left')
topic_classification = topic_classification.merge(video_description[['videoId', 'title']], left_on='VideoId', right_on='videoId', how='left')
topic_classification_passer = topic_classification.copy()
topic_classification

,VideoId,Topic,Probaibility,ReducedEmbeddings_x,ReducedEmbeddings_y,CustomName,videoId,title
0,09zYzvKkCOA,6,0.960030,2.788041,2.322691,Inspirational True Stories,09zYzvKkCOA,Three Christs - Official Trailer I HD I IFC Films
1,Na6gA1RehsU,0,1.000000,7.812714,3.023268,Movie Trailers,Na6gA1RehsU,TÁR - Official Trailer [HD] - In Select Theate...
2,gSMxBLlA8qY,17,0.673917,5.310845,2.776821,True Crime Dramas,gSMxBLlA8qY,RICHARD JEWELL - Official Trailer [HD]
3,tSkv_sEyPSc,5,0.478666,5.177978,3.813516,Uncovering Painful Truths,tSkv_sEyPSc,MENDING THE LINE | Official Trailer
4,b6qSGKN42Gc,3,1.000000,2.385987,3.115065,Movies and their themes,b6qSGKN42Gc,SILO - Official Trailer - Oscilloscope Laborat...
...,...,...,...,...,...,...,...,...
549,WuClROtJCaw,6,1.000000,2.976135,2.231939,Inspirational True Stories,WuClROtJCaw,ANOTHER ROUND Trailer [HD] Mongrel Media
550,uYPbbksJxIg,18,0.591520,3.114096,4.091232,Emotional Journeys Through Life and History,uYPbbksJxIg,Oppenheimer | New Trailer
551,P_dfc0iqmig,0,1.000000,6.612958,2.591962,Movie Trailers,P_dfc0iqmig,Hustlers | Official Trailer [HD] | Own it Now ...
552,j5BnpaiMvO4,0,1.000000,8.060444,4.137704,Movie Trailers,j5BnpaiMvO4,MICKEY AND THE BEAR Trailer (2019) Drama Movie


In [35]:
comments_ = comments_.loc[comments_['pos-neg'] != np.inf]
final = topic_classification.merge(comments_, left_on='VideoId', right_on='videoId', how='left')
final = final.loc[final['pos-neg'] > 0]
final['ReducedEmbeddings'] = [[i, j] for i, j in zip(final['ReducedEmbeddings_x'], final['ReducedEmbeddings_y'])]

generated_description = pd.read_csv(path+'/Data/Processed/ProcessedVideoDescription.csv',  sep='|', lineterminator='%')
generated_description = generated_description.sort_values('CreatedTime', ascending=False).drop_duplicates('VideoId', keep='first').reset_index(drop=True)
final = final.merge(generated_description[['VideoId', 'OpenAIGeneratedDescription', ]], left_on='VideoId', right_on='VideoId', how='left')
final

,VideoId,Topic,Probaibility,ReducedEmbeddings_x,ReducedEmbeddings_y,CustomName,videoId_x,title,videoId_y,positive_sent,negative_sent,neutral_sent,total_comments,pos-neg,ReducedEmbeddings,OpenAIGeneratedDescription
0,Na6gA1RehsU,0,1.000000,7.812714,3.023268,Movie Trailers,Na6gA1RehsU,TÁR - Official Trailer [HD] - In Select Theate...,Na6gA1RehsU,102.0,13.0,48.0,179.0,1.803144,"[7.812714, 3.0232685]",TÁR is a captivating and visually stunning fil...
1,gSMxBLlA8qY,17,0.673917,5.310845,2.776821,True Crime Dramas,gSMxBLlA8qY,RICHARD JEWELL - Official Trailer [HD],gSMxBLlA8qY,471.0,193.0,273.0,1086.0,1.169527,"[5.310845, 2.7768207]","In the upcoming film ""Richard Jewell,"" directe..."
2,nrXlY6gzTTM,20,0.312142,1.904737,2.963583,True stories of historical figures,nrXlY6gzTTM,The Post | Official Trailer [HD] | 20th Centur...,nrXlY6gzTTM,371.0,55.0,254.0,764.0,1.476344,"[1.9047374, 2.963583]","In this thrilling drama, legendary filmmaker S..."
3,XFYWazblaUA,18,0.760441,2.787400,3.801875,Emotional Journeys Through Life and History,XFYWazblaUA,THE SHAPE OF WATER | Official Trailer | FOX Se...,XFYWazblaUA,1181.0,224.0,1168.0,2826.0,1.307202,"[2.7874, 3.8018749]","Introducing ""The Shape of Water,"" a captivatin..."
4,-Denciie5oA,3,0.990479,2.245054,2.972476,Movies and their themes,-Denciie5oA,12 STRONG - Official Trailer,-Denciie5oA,430.0,256.0,441.0,1291.0,1.093524,"[2.245054, 2.9724758]",Get ready for an intense and action-packed rid...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,AHmCH7iB_IM,0,1.000000,6.514732,2.210091,Movie Trailers,AHmCH7iB_IM,CREED III | Official Trailer,AHmCH7iB_IM,2989.0,335.0,3017.0,7011.0,1.376421,"[6.514732, 2.2100906]","In the official trailer for ""Creed III,"" direc..."
241,bRDuc3IdOn8,15,1.000000,6.107909,3.331384,Mystery and Suspense Movies,bRDuc3IdOn8,The Painted Bird - Official Trailer I HD I IFC...,bRDuc3IdOn8,17.0,3.0,4.0,27.0,2.578902,"[6.1079087, 3.331384]",Experience the gripping and visually stunning ...
242,eYfKlNBLLeQ,22,0.671339,4.078673,2.577462,Emotional Journeys,eYfKlNBLLeQ,Little Joe - Official Trailer,eYfKlNBLLeQ,12.0,3.0,9.0,26.0,2.261860,"[4.078673, 2.577462]","In the movie trailer for ""Little Joe,"" we are ..."
243,uYPbbksJxIg,18,0.591520,3.114096,4.091232,Emotional Journeys Through Life and History,uYPbbksJxIg,Oppenheimer | New Trailer,uYPbbksJxIg,4537.0,2017.0,5287.0,13348.0,1.106534,"[3.1140964, 4.091232]",Get ready for a thrilling journey through hist...


In [60]:
final.loc[final['CustomName'] == 'Movie Synopses']

,VideoId,Topic,Probaibility,ReducedEmbeddings_x,ReducedEmbeddings_y,CustomName,videoId_x,title,videoId_y,positive_sent,negative_sent,neutral_sent,total_comments,pos-neg,ReducedEmbeddings,OpenAIGeneratedDescription
55,ZKxzlNE9_7Y,19,1.000000,4.070899,3.361918,Movie Synopses,ZKxzlNE9_7Y,The Breadwinner [Official US Trailer],ZKxzlNE9_7Y,8.0,5.0,4.0,18.0,1.292030,"[4.0708985, 3.3619175]","In a world fraught with darkness, one young gi..."
152,Q9hNIzP6ycM,19,1.000000,4.177200,3.486436,Movie Synopses,Q9hNIzP6ycM,Joe Bell | Official Trailer | In Theaters J...,Q9hNIzP6ycM,52.0,13.0,19.0,93.0,1.540476,"[4.1772003, 3.486436]","In the bustling streets of New York City, a yo..."
232,5RR8WTQzwSk,19,0.881466,4.109355,3.507395,Movie Synopses,5RR8WTQzwSk,Honey Boy - Official Trailer | Amazon Studios,5RR8WTQzwSk,107.0,8.0,93.0,223.0,2.247156,"[4.1093554, 3.5073946]","In this heartwarming and inspiring film, we fo..."


In [56]:
a = final.groupby('CustomName').agg(
    avg_pos_neg = pd.NamedAgg('pos-neg', aggfunc='mean'),
    count = pd.NamedAgg('total_comments', aggfunc='sum')
).sort_values('avg_pos_neg', ascending=False)
a = a.reset_index(drop=False)
a.sort_values('count', ascending=True)

,CustomName,avg_pos_neg,count
13,Romantic Comedies,1.855767,118.0
3,Adoption and Family Dynamics,2.452994,129.0
20,Movie Synopses,1.693221,334.0
26,Movies and their stars,1.544845,411.0
21,Innovators and Their Dreams,1.690528,425.0
1,Emotional Journeys,2.725802,489.0
0,Movies and Literature,3.814640,504.0
12,Heartwarming Friendship Stories,1.892559,1015.0
16,Uncovering Painful Truths,1.813434,1497.0
27,Biographies and Historical Dramas,1.428354,2040.0


In [40]:
px.scatter(final, x='ReducedEmbeddings_x', y='ReducedEmbeddings_y', width=1200, height=750, hover_data=['title', 'CustomName'], color='pos-neg')

In [ ]:
px.scatter(final, x='ReducedEmbeddings_x', y='ReducedEmbeddings_y', width=1200, height=750, hover_data=['title', 'pos-neg'], color='CustomName')

In [ ]:
final.to_csv('final_reg_data.csv', index=False)

In [ ]:
topic_classification_passer

,VideoId,Topic,Probaibility,ReducedEmbeddings_x,ReducedEmbeddings_y,CustomName,videoId,title
0,09zYzvKkCOA,6,0.960030,2.788041,2.322691,Inspirational True Stories,09zYzvKkCOA,Three Christs - Official Trailer I HD I IFC Films
1,Na6gA1RehsU,0,1.000000,7.812714,3.023268,Movie Trailers,Na6gA1RehsU,TÁR - Official Trailer [HD] - In Select Theate...
2,gSMxBLlA8qY,17,0.673917,5.310845,2.776821,True Crime Dramas,gSMxBLlA8qY,RICHARD JEWELL - Official Trailer [HD]
3,tSkv_sEyPSc,5,0.478666,5.177978,3.813516,Uncovering Painful Truths,tSkv_sEyPSc,MENDING THE LINE | Official Trailer
4,b6qSGKN42Gc,3,1.000000,2.385987,3.115065,Movies and their themes,b6qSGKN42Gc,SILO - Official Trailer - Oscilloscope Laborat...
...,...,...,...,...,...,...,...,...
549,WuClROtJCaw,6,1.000000,2.976135,2.231939,Inspirational True Stories,WuClROtJCaw,ANOTHER ROUND Trailer [HD] Mongrel Media
550,uYPbbksJxIg,18,0.591520,3.114096,4.091232,Emotional Journeys Through Life and History,uYPbbksJxIg,Oppenheimer | New Trailer
551,P_dfc0iqmig,0,1.000000,6.612958,2.591962,Movie Trailers,P_dfc0iqmig,Hustlers | Official Trailer [HD] | Own it Now ...
552,j5BnpaiMvO4,0,1.000000,8.060444,4.137704,Movie Trailers,j5BnpaiMvO4,MICKEY AND THE BEAR Trailer (2019) Drama Movie


In [ ]:
commets__ = comments[['videoId', 'comment', 'sentiment']].merge(topic_classification_passer.drop(['videoId'], axis=1), left_on='videoId', right_on='VideoId', how='left')

commets__ = commets__.groupby('CustomName').agg(
    positive_sent = pd.NamedAgg('sentiment', aggfunc=lambda x: (x == 'positive').sum()),
    negative_sent = pd.NamedAgg('sentiment', aggfunc=lambda x: (x == 'negative').sum()),
    neutral_sent = pd.NamedAgg('sentiment', aggfunc=lambda x: (x == 'neutral').sum()),
    total_comments = pd.NamedAgg('videoId', aggfunc='count')
).reset_index(drop=False)

commets__['pos-neg'] = np.log(comments_['positive_sent']) / np.log(comments_['negative_sent'])
commets__

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pandas/core/arraylike.py:396: RuntimeWarning:

divide by zero encountered in log



,CustomName,positive_sent,negative_sent,neutral_sent,total_comments,pos-neg
0,Adoption and Family Dynamics,79,8,52,146,2.929947
1,Biographies and Historical Dramas,1039,165,697,2094,-0.000000
2,Coming-of-age films,2836,420,2313,6157,2.020559
3,Emotional Journeys,230,70,156,495,1.093524
4,Emotional Journeys Through Life and History,5754,2244,6472,16231,1.470747
5,Empowering Stories of Resilience,5640,887,3894,11509,1.278464
6,Heartwarming Friendship Stories,414,71,461,1062,-0.000000
7,Heartwarming Movie Releases,1344,194,1132,3052,-0.000000
8,Horror and Ghost Stories,3224,799,3495,8529,-0.000000
9,Innovators and Their Dreams,205,36,155,426,-0.000000


In [ ]:
commets__['pos_neg_mod'] = [i if i>0 else 0 for i in commets__['pos-neg']]
commets__

,CustomName,positive_sent,negative_sent,neutral_sent,total_comments,pos-neg,pos_neg_mod
0,Adoption and Family Dynamics,79,8,52,146,2.929947,2.929947
1,Biographies and Historical Dramas,1039,165,697,2094,-0.000000,0.000000
2,Coming-of-age films,2836,420,2313,6157,2.020559,2.020559
3,Emotional Journeys,230,70,156,495,1.093524,1.093524
4,Emotional Journeys Through Life and History,5754,2244,6472,16231,1.470747,1.470747
5,Empowering Stories of Resilience,5640,887,3894,11509,1.278464,1.278464
6,Heartwarming Friendship Stories,414,71,461,1062,-0.000000,0.000000
7,Heartwarming Movie Releases,1344,194,1132,3052,-0.000000,0.000000
8,Horror and Ghost Stories,3224,799,3495,8529,-0.000000,0.000000
9,Innovators and Their Dreams,205,36,155,426,-0.000000,0.000000


In [ ]:
final2 = final.merge(commets__[['CustomName', 'pos_neg_mod']], left_on=['CustomName'], right_on=['CustomName'], how='left')
final2

,VideoId,Topic,Probaibility,ReducedEmbeddings_x,ReducedEmbeddings_y,CustomName,videoId_x,title,videoId_y,positive_sent,negative_sent,neutral_sent,total_comments,pos-neg,ReducedEmbeddings,OpenAIGeneratedDescription,pos_neg_mod
0,Na6gA1RehsU,0,1.000000,7.812714,3.023268,Movie Trailers,Na6gA1RehsU,TÁR - Official Trailer [HD] - In Select Theate...,Na6gA1RehsU,102.0,13.0,48.0,179.0,1.803144,"[7.812714, 3.0232685]",TÁR is a captivating and visually stunning fil...,0.000000
1,gSMxBLlA8qY,17,0.673917,5.310845,2.776821,True Crime Dramas,gSMxBLlA8qY,RICHARD JEWELL - Official Trailer [HD],gSMxBLlA8qY,471.0,193.0,273.0,1086.0,1.169527,"[5.310845, 2.7768207]","In the upcoming film ""Richard Jewell,"" directe...",1.310565
2,nrXlY6gzTTM,20,0.312142,1.904737,2.963583,True stories of historical figures,nrXlY6gzTTM,The Post | Official Trailer [HD] | 20th Centur...,nrXlY6gzTTM,371.0,55.0,254.0,764.0,1.476344,"[1.9047374, 2.963583]","In this thrilling drama, legendary filmmaker S...",2.000000
3,XFYWazblaUA,18,0.760441,2.787400,3.801875,Emotional Journeys Through Life and History,XFYWazblaUA,THE SHAPE OF WATER | Official Trailer | FOX Se...,XFYWazblaUA,1181.0,224.0,1168.0,2826.0,1.307202,"[2.7874, 3.8018749]","Introducing ""The Shape of Water,"" a captivatin...",1.470747
4,-Denciie5oA,3,0.990479,2.245054,2.972476,Movies and their themes,-Denciie5oA,12 STRONG - Official Trailer,-Denciie5oA,430.0,256.0,441.0,1291.0,1.093524,"[2.245054, 2.9724758]",Get ready for an intense and action-packed rid...,1.358033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,AHmCH7iB_IM,0,1.000000,6.514732,2.210091,Movie Trailers,AHmCH7iB_IM,CREED III | Official Trailer,AHmCH7iB_IM,2989.0,335.0,3017.0,7011.0,1.376421,"[6.514732, 2.2100906]","In the official trailer for ""Creed III,"" direc...",0.000000
241,bRDuc3IdOn8,15,1.000000,6.107909,3.331384,Mystery and Suspense Movies,bRDuc3IdOn8,The Painted Bird - Official Trailer I HD I IFC...,bRDuc3IdOn8,17.0,3.0,4.0,27.0,2.578902,"[6.1079087, 3.331384]",Experience the gripping and visually stunning ...,1.366309
242,eYfKlNBLLeQ,22,0.671339,4.078673,2.577462,Emotional Journeys,eYfKlNBLLeQ,Little Joe - Official Trailer,eYfKlNBLLeQ,12.0,3.0,9.0,26.0,2.261860,"[4.078673, 2.577462]","In the movie trailer for ""Little Joe,"" we are ...",1.093524
243,uYPbbksJxIg,18,0.591520,3.114096,4.091232,Emotional Journeys Through Life and History,uYPbbksJxIg,Oppenheimer | New Trailer,uYPbbksJxIg,4537.0,2017.0,5287.0,13348.0,1.106534,"[3.1140964, 4.091232]",Get ready for a thrilling journey through hist...,1.470747


In [ ]:
px.scatter(final2, x='ReducedEmbeddings_x', y='ReducedEmbeddings_y', width=1200, height=750, hover_data=['title', 'CustomName'], color='pos_neg_mod')

In [ ]:
final.loc[final['title'] == 'Murder on the Orient Express | Official Trailer [HD] | 20th Century FOX']

,VideoId,Topic,Probaibility,ReducedEmbeddings_x,ReducedEmbeddings_y,CustomName,videoId_x,title,videoId_y,positive_sent,negative_sent,neutral_sent,total_comments,pos-neg,ReducedEmbeddings,OpenAIGeneratedDescription
33,Mq4m3yAoW8E,14,1.0,4.78656,2.202778,Movies and Truth,Mq4m3yAoW8E,Murder on the Orient Express | Official Traile...,Mq4m3yAoW8E,1563.0,484.0,1303.0,3772.0,1.189625,"[4.78656, 2.2027779]","Experience the breathtaking journey of ""Murder..."


In [ ]:
comments.loc[comments['videoId'] == 'Mq4m3yAoW8E']

,videoId,code,error_reason,id,replies,comment,postedAt,publishedAt,cutoffdate,score,negative,neutral,positive,compound,sentiment
304267,Mq4m3yAoW8E,200,NaN,UghNUCVn1vGe_XgCoAEC,0.0,johnny depp penelope cruz awesome film,2017-06-22 12:57:51+00:00,2017-06-01 13:00:34+00:00,2017-06-22 13:00:34+00:00,"{'neg': 0.0, 'neu': 0.549, 'pos': 0.451, 'comp...",0.000,0.549,0.451,0.6249,positive
304268,Mq4m3yAoW8E,200,NaN,UghfmllpF5mNnngCoAEC,0.0,waiting say i ve read book movie,2017-06-22 10:41:36+00:00,2017-06-01 13:00:34+00:00,2017-06-22 13:00:34+00:00,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,neutral
304269,Mq4m3yAoW8E,200,NaN,Ugh1iotOB3QueXgCoAEC,0.0,read book summer and climax struck surprise ...,2017-06-22 09:04:46+00:00,2017-06-01 13:00:34+00:00,2017-06-22 13:00:34+00:00,"{'neg': 0.134, 'neu': 0.47, 'pos': 0.396, 'com...",0.134,0.470,0.396,0.5994,positive
304270,Mq4m3yAoW8E,200,NaN,UgiwS87OE3-YO3gCoAEC,0.0,colonel arbuthnot,2017-06-22 03:18:17+00:00,2017-06-01 13:00:34+00:00,2017-06-22 13:00:34+00:00,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,neutral
304271,Mq4m3yAoW8E,200,NaN,Ugj3v4gupZAR_XgCoAEC,0.0,good song go trailer instead believer human ...,2017-06-22 02:15:13+00:00,2017-06-01 13:00:34+00:00,2017-06-22 13:00:34+00:00,"{'neg': 0.124, 'neu': 0.725, 'pos': 0.15, 'com...",0.124,0.725,0.150,0.1280,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308034,Mq4m3yAoW8E,200,NaN,UggcI6IH82J1EngCoAEC,0.0,sub nd sub,2017-06-01 13:05:39+00:00,2017-06-01 13:00:34+00:00,2017-06-22 13:00:34+00:00,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,neutral
308035,Mq4m3yAoW8E,200,NaN,UgjFt0C9KxWotHgCoAEC,0.0,cool,2017-06-01 13:05:12+00:00,2017-06-01 13:00:34+00:00,2017-06-22 13:00:34+00:00,"{'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound...",0.000,0.000,1.000,0.3182,positive
308036,Mq4m3yAoW8E,200,NaN,UgjzA_xQKnvyS3gCoAEC,0.0,stylelized perfect,2017-06-01 13:03:21+00:00,2017-06-01 13:00:34+00:00,2017-06-22 13:00:34+00:00,"{'neg': 0.0, 'neu': 0.213, 'pos': 0.787, 'comp...",0.000,0.213,0.787,0.5719,positive
308037,Mq4m3yAoW8E,200,NaN,UgjFC_IxHt3SrngCoAEC,0.0,100 club,2017-06-01 13:03:07+00:00,2017-06-01 13:00:34+00:00,2017-06-22 13:00:34+00:00,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.000,1.000,0.000,0.0000,neutral
